In [1]:
# http://www.wildml.com/2016/08/rnns-in-tensorflow-a-practical-guide-and-undocumented-features/
# http://learningtensorflow.com/index.html
# http://suriyadeepan.github.io/2016-12-31-practical-seq2seq/

import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [2]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

# One cell RNN input_dim (3) -> output_dim (5)
hidden_size = 5
cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
print(cell.output_size, cell.state_size)

x_data = np.array([[[1, 2, 3]]], dtype=np.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval())
sess.close()

5 5
array([[[ 0.12514158, -0.85017031,  0.71309912,  0.03091768,  0.85316288]]], dtype=float32)


In [3]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

# One cell RNN input_dim (3) -> output_dim (5). sequence: 2
hidden_size = 5
cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
x_data = np.array([[[1, 2, 3],
                    [4, 5, 6]]], dtype=np.float32)
outputs, states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval())
sess.close()

array([[[ 0.10406052, -0.5249666 ,  0.92754203, -0.93592882, -0.92667967],
        [-0.56909806, -0.97837865,  0.9967671 , -0.99993688, -0.99961668]]], dtype=float32)


In [4]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
# One cell RNN input_dim (3) -> output_dim (5). sequence: 2
cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
x_data = np.array([[[1, 2, 3],
                    [4, 5, 6]],

                   [[7, 8, 9],
                    [10, 11, 12]],

                   [[13, 14, 15],
                    [16, 17, 18]], ], dtype=np.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, x_data, sequence_length=[1,2,1], dtype=tf.float32)
sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval())
sess.close()

array([[[ -1.89281225e-01,  -1.35549843e-01,   4.49652344e-01,
          -8.63934755e-02,   2.45035127e-01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]],

       [[ -9.32960287e-02,  -4.47449461e-03,   6.99320793e-01,
          -1.28612280e-01,   5.51817417e-01],
        [ -4.36719432e-02,  -5.55863325e-03,   9.35384154e-01,
          -1.25395089e-01,   8.74094665e-01]],

       [[ -2.91416775e-02,  -1.08962704e-04,   7.35959411e-01,
          -9.44481641e-02,   6.81338310e-01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]]], dtype=float32)


In [5]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
# One cell RNN input_dim (3) -> output_dim (5). sequence: 2

batch_size = 3
cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)

x_data = np.array([[[1, 2, 3],
                    [4, 5, 6]],

                   [[7, 8, 9],
                    [10, 11, 12]],

                   [[13, 14, 15],
                    [16, 17, 18]], ], dtype=np.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, x_data,
                                     initial_state=initial_state, dtype=tf.float32)
sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval())
sess.close()

array([[[ -9.33315419e-03,   1.37675563e-02,  -4.86772433e-02,
          -1.69695914e-02,   2.20286071e-01],
        [ -3.43358633e-03,  -5.50280809e-02,  -9.33276583e-03,
           2.18467321e-02,   4.00748491e-01]],

       [[ -2.15095861e-05,  -5.14130257e-02,  -2.82033870e-04,
           8.88978504e-03,   1.84698686e-01],
        [ -3.24986604e-06,  -2.85751894e-02,  -4.61524760e-05,
           2.65101274e-03,   3.17526817e-01]],

       [[ -1.03909752e-08,  -1.13779102e-02,  -9.16522595e-07,
           5.06423879e-04,   9.83619094e-02],
        [ -7.08550218e-10,  -4.92142467e-03,  -1.53678755e-07,
           1.12846312e-04,   1.69380605e-01]]], dtype=float32)


In [6]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
# Create input data
x_data = np.arange(24, dtype=np.float32).reshape(2, 4, 3)
pp.pprint(x_data)  # batch, sequence_length, input size

array([[[  0.,   1.,   2.],
        [  3.,   4.,   5.],
        [  6.,   7.,   8.],
        [  9.,  10.,  11.]],

       [[ 12.,  13.,  14.],
        [ 15.,  16.,  17.],
        [ 18.,  19.,  20.],
        [ 21.,  22.,  23.]]], dtype=float32)


In [7]:
# Make rnn
cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
cell = rnn.MultiRNNCell([cell] * 3, state_is_tuple=True)

# rnn in/out
outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
print("dynamic rnn: ", outputs)
sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval())  # batch size, unrolling (time), hidden_size

dynamic rnn:  Tensor("rnn/transpose:0", shape=(2, 4, 5), dtype=float32)
array([[[-0.00459586, -0.00049015,  0.00231713,  0.00150267, -0.00309547],
        [-0.01405779, -0.00275343,  0.00728929,  0.00569544, -0.00943902],
        [-0.02541408, -0.00688672,  0.01274858,  0.01187734, -0.01676471],
        [-0.03622811, -0.01255005,  0.01701238,  0.0190992 , -0.02300376]],

       [[-0.00146927,  0.0005761 ,  0.00076853,  0.00075077, -0.00244083],
        [-0.00370315,  0.00137755,  0.00149405,  0.00206371, -0.00615409],
        [-0.00608444,  0.0020358 ,  0.00185139,  0.00371246, -0.01002057],
        [-0.00822647,  0.00236944,  0.00178765,  0.00550162, -0.01337252]]], dtype=float32)


In [8]:
cell = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32,
                                     sequence_length=[1, 3]) # lentgh 1 for batch 1, lentgh 2 for batch 2
print("dynamic rnn: ", outputs)
sess.run(tf.global_variables_initializer())
pp.pprint(outputs.eval())  # batch size, unrolling (time), hidden_size
sess.close()

dynamic rnn:  Tensor("rnn_1/transpose:0", shape=(2, 4, 5), dtype=float32)
array([[[ -8.16378146e-02,  -8.35597441e-02,  -8.39958861e-02,
           2.73834332e-03,  -7.95639902e-02],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]],

       [[ -1.31173534e-08,  -5.95647361e-05,  -1.94320809e-02,
          -4.60592355e-06,   5.68799749e-02],
        [ -1.53974222e-09,  -6.10628103e-06,  -9.22685955e-03,
          -3.11785243e-07,   1.10334031e-01],
        [ -1.62557925e-10,  -6.19913749e-07,  -4.28946177e-03,
          -1.87093434e-08,   1.44816950e-01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]]], dtype=float32)


In [9]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
# bi-directional rnn
cell_fw = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)
cell_bw = rnn.BasicLSTMCell(num_units=5, state_is_tuple=True)

outputs, states = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, x_data,
                                                  sequence_length=[2, 3],
                                                  dtype=tf.float32)

sess.run(tf.global_variables_initializer())
pp.pprint(sess.run(outputs))
pp.pprint(sess.run(states))

(   array([[[  8.31724256e-02,  -1.90516692e-02,  -1.20053247e-01,
           1.73350230e-01,  -2.21191987e-01],
        [  3.95709813e-01,   3.11785251e-01,  -1.59265280e-01,
           6.85954273e-01,  -4.07920510e-01],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]],

       [[  6.45857632e-01,   3.68143737e-01,  -4.07030951e-04,
           7.58043289e-01,  -1.89784449e-02],
        [  8.58740807e-01,   4.98793900e-01,  -3.45722307e-04,
           9.63332772e-01,  -2.57388130e-02],
        [  9.17318940e-01,   4.97768909e-01,  -2.52985512e-04,
           9.94926691e-01,  -2.32869536e-02],
        [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00]]], dtype=float32),
    array([[[  8.28782916e-02,  -4.70337570e-02,   1.39396042e-01,
           1.59858704e-01,  -3.745330

In [10]:
# flattern based softmax
hidden_size=3
sequence_length=4
batch_size=2
num_classes=5

pp.pprint(x_data) # hidden_size=3, sequence_length=4, batch_size=2
x_data = x_data.reshape(-1, hidden_size)
pp.pprint(x_data)

softmax_w = np.arange(15, dtype=np.float32).reshape(hidden_size, num_classes)
outputs = np.matmul(x_data, softmax_w)
outputs = outputs.reshape(-1, sequence_length, num_classes) # batch, seq, class
pp.pprint(outputs)

array([[[  0.,   1.,   2.],
        [  3.,   4.,   5.],
        [  6.,   7.,   8.],
        [  9.,  10.,  11.]],

       [[ 12.,  13.,  14.],
        [ 15.,  16.,  17.],
        [ 18.,  19.,  20.],
        [ 21.,  22.,  23.]]], dtype=float32)
array([[  0.,   1.,   2.],
       [  3.,   4.,   5.],
       [  6.,   7.,   8.],
       [  9.,  10.,  11.],
       [ 12.,  13.,  14.],
       [ 15.,  16.,  17.],
       [ 18.,  19.,  20.],
       [ 21.,  22.,  23.]], dtype=float32)
array([[[  25.,   28.,   31.,   34.,   37.],
        [  70.,   82.,   94.,  106.,  118.],
        [ 115.,  136.,  157.,  178.,  199.],
        [ 160.,  190.,  220.,  250.,  280.]],

       [[ 205.,  244.,  283.,  322.,  361.],
        [ 250.,  298.,  346.,  394.,  442.],
        [ 295.,  352.,  409.,  466.,  523.],
        [ 340.,  406.,  472.,  538.,  604.]]], dtype=float32)


In [11]:
# [batch_size, sequence_length, emb_dim ]
prediction1 = tf.constant([[[0, 1], [0, 1], [0, 1]]], dtype=tf.float32)
prediction2 = tf.constant([[[1, 0], [1, 0], [1, 0]]], dtype=tf.float32)
prediction3 = tf.constant([[[0, 1], [1, 0], [0, 1]]], dtype=tf.float32)

# [batch_size, sequence_length]
y_data = tf.constant([[1, 1, 1]])

# [batch_size * sequence_length]
weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

sequence_loss1 = tf.contrib.seq2seq.sequence_loss(prediction1, y_data, weights)
sequence_loss2 = tf.contrib.seq2seq.sequence_loss(prediction2, y_data, weights)
sequence_loss3 = tf.contrib.seq2seq.sequence_loss(prediction3, y_data, weights)

sess.run(tf.global_variables_initializer())
print("Loss1: ", sequence_loss1.eval(),
      "Loss2: ", sequence_loss2.eval(),
      "Loss3: ", sequence_loss3.eval())

Loss1:  0.313262 Loss2:  1.31326 Loss3:  0.646595
